# Análise de feedbacks

Análise dos comentários presentes nas base 'barber_shop'.

## Extração de dados

In [1]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
import os

load_dotenv()

True

In [2]:
user = os.getenv('user')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')
ssl_cert= '../configs/ca.crt'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}?sslmode=verify-full&sslrootcert={ssl_cert}')

In [3]:
with open('../sql_queries/select_feedbacks.sql', 'r') as file:
    query = file.read()

data = pd.read_sql(query, engine)

In [4]:
data

,dat_comp,av_comentario,nm_cliente,nm_funcionario,tp_servico
0,2022-02-01,"Resultado aceitável, mas esperava mais.",Catarina Rocha,Maria Oliveira,Alisamento
1,2022-04-01,"Não gostei, o profissional foi desatento e o c...",Anthony da Mata,João Silva,Alisamento
2,2022-03-01,"Péssimo serviço, não volto mais.",Kamilly Santos,Maria Oliveira,Alisamento
3,2021-12-01,"O resultado não foi satisfatório, poderia ser ...",Kamilly Gonçalves,Maria Oliveira,Alisamento
4,2021-07-01,"Foi bom, mas o atendimento precisa melhorar.",Heloísa Correia,Carlos Souza,Alisamento
...,...,...,...,...,...
2995,2023-06-01,"Não gostei muito, não ficou como esperado.",Enzo Gomes,Ana Lima,Tratamento capilar
2996,2023-08-01,"Corte bom, voltarei mais vezes.",Lucca Ferreira,João Silva,Tratamento capilar
2997,2022-06-01,"Corte horrível, totalmente diferente do que pedi.",Rafaela da Rosa,Ana Lima,Tratamento capilar
2998,2022-09-01,"Bom serviço, mas poderia ser um pouco mais ráp...",Helena Melo,Ana Lima,Tratamento capilar


# Pré-processamento do texto

Cada comentário define um corpus

## Tokenização

Tokenizando cada comentário individualmente

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
data['tokens'] = data['av_comentario'].apply(
    lambda comentario: nltk.word_tokenize(comentario.lower(), language= 'portuguese')
    )

### Remoção de stopwords e palavras do negócio

In [7]:
from nltk.corpus import stopwords

In [8]:
data['tokens_sem_stopwords'] = data['tokens'].apply(
    lambda tokens: [token for token in tokens if token not in stopwords.words('portuguese')]
)

In [9]:
business_words = ['corte', 'serviço', 'atendimento']

data['tokens_sem_stopwords'] = data['tokens_sem_stopwords'].apply(
    lambda tokens: [token for token in tokens if token not in business_words]
)

### Remoção de números

In [10]:
import re

In [11]:
data['tokens_sem_stopwords_sem_numeros'] = data['tokens_sem_stopwords'].apply(
    lambda tokens: [re.sub('\d+', '', token) for token in tokens]
)

### Remoção de pontuação

In [12]:
import string

In [13]:
data['tokens_sem_stopwords_sem_numeros_sem_pontuacao'] = data['tokens_sem_stopwords_sem_numeros'].apply(
    lambda tokens: [token for token in tokens if token not in string.punctuation]
)

In [14]:
data

,dat_comp,av_comentario,nm_cliente,nm_funcionario,tp_servico,tokens,tokens_sem_stopwords,tokens_sem_stopwords_sem_numeros,tokens_sem_stopwords_sem_numeros_sem_pontuacao
0,2022-02-01,"Resultado aceitável, mas esperava mais.",Catarina Rocha,Maria Oliveira,Alisamento,"[resultado, aceitável, ,, mas, esperava, mais, .]","[resultado, aceitável, ,, esperava, .]","[resultado, aceitável, ,, esperava, .]","[resultado, aceitável, esperava]"
1,2022-04-01,"Não gostei, o profissional foi desatento e o c...",Anthony da Mata,João Silva,Alisamento,"[não, gostei, ,, o, profissional, foi, desaten...","[gostei, ,, profissional, desatento, ficou, pé...","[gostei, ,, profissional, desatento, ficou, pé...","[gostei, profissional, desatento, ficou, péssimo]"
2,2022-03-01,"Péssimo serviço, não volto mais.",Kamilly Santos,Maria Oliveira,Alisamento,"[péssimo, serviço, ,, não, volto, mais, .]","[péssimo, ,, volto, .]","[péssimo, ,, volto, .]","[péssimo, volto]"
3,2021-12-01,"O resultado não foi satisfatório, poderia ser ...",Kamilly Gonçalves,Maria Oliveira,Alisamento,"[o, resultado, não, foi, satisfatório, ,, pode...","[resultado, satisfatório, ,, poderia, melhor, .]","[resultado, satisfatório, ,, poderia, melhor, .]","[resultado, satisfatório, poderia, melhor]"
4,2021-07-01,"Foi bom, mas o atendimento precisa melhorar.",Heloísa Correia,Carlos Souza,Alisamento,"[foi, bom, ,, mas, o, atendimento, precisa, me...","[bom, ,, precisa, melhorar, .]","[bom, ,, precisa, melhorar, .]","[bom, precisa, melhorar]"
...,...,...,...,...,...,...,...,...,...
2995,2023-06-01,"Não gostei muito, não ficou como esperado.",Enzo Gomes,Ana Lima,Tratamento capilar,"[não, gostei, muito, ,, não, ficou, como, espe...","[gostei, ,, ficou, esperado, .]","[gostei, ,, ficou, esperado, .]","[gostei, ficou, esperado]"
2996,2023-08-01,"Corte bom, voltarei mais vezes.",Lucca Ferreira,João Silva,Tratamento capilar,"[corte, bom, ,, voltarei, mais, vezes, .]","[bom, ,, voltarei, vezes, .]","[bom, ,, voltarei, vezes, .]","[bom, voltarei, vezes]"
2997,2022-06-01,"Corte horrível, totalmente diferente do que pedi.",Rafaela da Rosa,Ana Lima,Tratamento capilar,"[corte, horrível, ,, totalmente, diferente, do...","[horrível, ,, totalmente, diferente, pedi, .]","[horrível, ,, totalmente, diferente, pedi, .]","[horrível, totalmente, diferente, pedi]"
2998,2022-09-01,"Bom serviço, mas poderia ser um pouco mais ráp...",Helena Melo,Ana Lima,Tratamento capilar,"[bom, serviço, ,, mas, poderia, ser, um, pouco...","[bom, ,, poderia, pouco, rápido, .]","[bom, ,, poderia, pouco, rápido, .]","[bom, poderia, pouco, rápido]"


# Visualização de palavras

Avaliar a frequência de palavras em cada cenário:

* por funcionário
* por serviço
* por cliente

## Por funcionário

In [15]:
data['nm_funcionario'].unique()

array(['Maria Oliveira', 'João Silva', 'Carlos Souza', 'Ana Lima'],
      dtype=object)

In [33]:
ana_lima_corpus = list(data[data['nm_funcionario'] == 'Ana Lima']['tokens_sem_stopwords_sem_numeros_sem_pontuacao'])

In [37]:
ana_lima_corpus

[['gostei', 'profissional', 'desatento', 'ficou', 'péssimo'],
 ['regular', 'precisa', 'atenção', 'detalhes'],
 ['resultado', 'satisfatório', 'poderia', 'melhor'],
 ['bom', 'voltarei', 'vezes'],
 ['péssimo', 'volto'],
 ['fraco', 'deixou', 'desejar'],
 ['ruim', 'fiquei', 'insatisfeito'],
 ['bom', 'poderia', 'pouco', 'rápido'],
 ['excelente', 'adorei'],
 ['gostei', 'pouco', 'demorado'],
 ['ótimo', 'resultado', 'apenas', 'detalhe', 'pode', 'melhorar'],
 ['gostei', 'pouco', 'demorado'],
 ['gostei', 'pouco', 'demorado'],
 ['excelente', 'adorei'],
 ['simplesmente', 'maravilhoso', 'nota'],
 ['pior', 'experiência', 'recomendo'],
 ['bom', 'poderia', 'pouco', 'rápido'],
 ['gostei', 'ficou', 'esperado'],
 ['razoável', 'impressionou'],
 ['bom', 'poderia', 'pouco', 'rápido'],
 ['horrível', 'totalmente', 'diferente', 'pedi'],
 ['excelente', 'adorei'],
 ['simplesmente', 'maravilhoso', 'nota'],
 ['excelente', 'adorei'],
 ['péssimo', 'volto'],
 ['gostei', 'ficou', 'esperado'],
 ['bom', 'voltarei', 'veze

In [43]:
from nltk.util import ngrams

n_grams = [list(ngrams(corpus, 2)) for corpus in ana_lima_corpus]

In [44]:
n_grams

[[('gostei', 'profissional'),
  ('profissional', 'desatento'),
  ('desatento', 'ficou'),
  ('ficou', 'péssimo')],
 [('regular', 'precisa'), ('precisa', 'atenção'), ('atenção', 'detalhes')],
 [('resultado', 'satisfatório'),
  ('satisfatório', 'poderia'),
  ('poderia', 'melhor')],
 [('bom', 'voltarei'), ('voltarei', 'vezes')],
 [('péssimo', 'volto')],
 [('fraco', 'deixou'), ('deixou', 'desejar')],
 [('ruim', 'fiquei'), ('fiquei', 'insatisfeito')],
 [('bom', 'poderia'), ('poderia', 'pouco'), ('pouco', 'rápido')],
 [('excelente', 'adorei')],
 [('gostei', 'pouco'), ('pouco', 'demorado')],
 [('ótimo', 'resultado'),
  ('resultado', 'apenas'),
  ('apenas', 'detalhe'),
  ('detalhe', 'pode'),
  ('pode', 'melhorar')],
 [('gostei', 'pouco'), ('pouco', 'demorado')],
 [('gostei', 'pouco'), ('pouco', 'demorado')],
 [('excelente', 'adorei')],
 [('simplesmente', 'maravilhoso'), ('maravilhoso', 'nota')],
 [('pior', 'experiência'), ('experiência', 'recomendo')],
 [('bom', 'poderia'), ('poderia', 'pouco')

In [35]:
for i in range(0, len(n_grams)):
    print(f'Bigrama {i}')
    print(n_grams[i])

Bigrama 0
[('gostei', 'profissional'), ('profissional', 'desatento'), ('desatento', 'ficou'), ('ficou', 'péssimo')]
Bigrama 1
[('regular', 'precisa'), ('precisa', 'atenção'), ('atenção', 'detalhes')]
Bigrama 2
[('resultado', 'satisfatório'), ('satisfatório', 'poderia'), ('poderia', 'melhor')]
Bigrama 3
[('bom', 'voltarei'), ('voltarei', 'vezes')]
Bigrama 4
[('péssimo', 'volto')]
Bigrama 5
[('fraco', 'deixou'), ('deixou', 'desejar')]
Bigrama 6
[('ruim', 'fiquei'), ('fiquei', 'insatisfeito')]
Bigrama 7
[('bom', 'poderia'), ('poderia', 'pouco'), ('pouco', 'rápido')]
Bigrama 8
[('excelente', 'adorei')]
Bigrama 9
[('gostei', 'pouco'), ('pouco', 'demorado')]
Bigrama 10
[('ótimo', 'resultado'), ('resultado', 'apenas'), ('apenas', 'detalhe'), ('detalhe', 'pode'), ('pode', 'melhorar')]
Bigrama 11
[('gostei', 'pouco'), ('pouco', 'demorado')]
Bigrama 12
[('gostei', 'pouco'), ('pouco', 'demorado')]
Bigrama 13
[('excelente', 'adorei')]
Bigrama 14
[('simplesmente', 'maravilhoso'), ('maravilhoso', 'n